**1.** En este bloque se importan las librerías necesarias (**pandas** y **numpy**) y se carga el archivo `A1.4 Vino Tinto.csv` con la función `pd.read_csv()`.  
Luego se utiliza `df.head()` para mostrar las primeras filas del DataFrame y verificar que los datos se hayan cargado correctamente.


In [30]:
import pandas as pd
import numpy as np

df = pd.read_csv("A1.4 Vino Tinto.csv")
print("Dimensiones (filas, columnas):", df.shape)
print(df.head())

Dimensiones (filas, columnas): (1599, 12)
   acidezFija  acidezVolatil  acidoCitrico  azucarResidual  cloruros  \
0         7.4           0.70          0.00             1.9     0.076   
1         7.8           0.88          0.00             2.6     0.098   
2         7.8           0.76          0.04             2.3     0.092   
3        11.2           0.28          0.56             1.9     0.075   
4         7.4           0.70          0.00             1.9     0.076   

   dioxidoAzufreLibre  dioxidoAzufreTotal  densidad    pH  sulfatos  alcohol  \
0                11.0                34.0    0.9978  3.51      0.56      9.4   
1                25.0                67.0    0.9968  3.20      0.68      9.8   
2                15.0                54.0    0.9970  3.26      0.65      9.8   
3                17.0                60.0    0.9980  3.16      0.58      9.8   
4                11.0                34.0    0.9978  3.51      0.56      9.4   

   calidad  
0        5  
1        5  
2    

**2.** En este bloque se separa la variable objetivo (**calidad**) de las variables predictoras.  
Después se utiliza `train_test_split` para dividir los datos en entrenamiento (80%) y prueba (20%), asegurando reproducibilidad con `random_state=42`.  
Finalmente, se imprimen la cantidad total de observaciones, el tamaño de cada conjunto y la lista de columnas usadas como características.

In [5]:
from sklearn.model_selection import train_test_split
Y = df["calidad"]
X = df.drop(columns = ["calidad"])
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state = 42)
print("Observaciones totales ", len(df))
print("Train ", X_train.shape[0], "- Test ", X_test.shape[0])
print("Columnas (features) ", list(X.columns))

Observaciones totales  1599
Train  1279 - Test  320
Columnas (features)  ['acidezFija', 'acidezVolatil', 'acidoCitrico', 'azucarResidual', 'cloruros', 'dioxidoAzufreLibre', 'dioxidoAzufreTotal', 'densidad', 'pH', 'sulfatos', 'alcohol']


**3.** En este bloque se utiliza un modelo de **regresión lineal** junto con la función `SequentialFeatureSelector` para realizar la **selección de características hacia adelante (forward)**.  
El selector va probando distintas combinaciones de variables entre 2 y 8 y escoge aquellas que maximizan la métrica **R²** con validación cruzada de 10 particiones.  
Como resultado, se muestran las variables seleccionadas, que en este caso fueron: `acidezVolatil`, `cloruros`, `dioxidoAzufreLibre`, `dioxidoAzufreTotal`, `pH`, `sulfatos` y `alcohol`.


In [8]:
from sklearn.linear_model import LinearRegression
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

lr = LinearRegression()

sfs_forward = SFS(
    estimator = lr,
    k_features=(2,8),
    forward = True,
    scoring = 'r2',
    cv = 10)

sfs_forward.fit(X_train.values, Y_train.values)
feat_forward = [X.columns[i] for i in sfs_forward.k_feature_idx_]

print("Variables seleccionadas con foward: ", feat_forward)

Variables seleccionadas con foward:  ['acidezVolatil', 'cloruros', 'dioxidoAzufreLibre', 'dioxidoAzufreTotal', 'pH', 'sulfatos', 'alcohol']


**4.** En este bloque se entrena un modelo de **regresión lineal** usando únicamente las variables seleccionadas por el método forward.  
Se realizan predicciones tanto sobre el conjunto de entrenamiento como en el de prueba y se calcula el **R²** con la función `r2_score`.  
Los resultados muestran un R² de aproximadamente **0.3477** en entrenamiento y **0.4013** en prueba, lo que indica que el modelo tiene un desempeño moderado y generaliza un poco mejor en los datos de prueba que en los de entrenamiento.


In [12]:
from sklearn.metrics import r2_score

lr_fwd = LinearRegression().fit(X_train[feat_forward], Y_train)

y_pred_train_fwd = lr_fwd.predict(X_train[feat_forward])
y_pred_test_fwd = lr_fwd.predict(X_test[feat_forward])

print("R² train (forward):", r2_score(Y_train, y_pred_train_fwd))
print("R² test (forward):", r2_score(Y_test, y_pred_test_fwd))

R² train (forward): 0.3474674883154586
R² test (forward): 0.40126288354402995


**5.** En este bloque se aplica la **selección de características hacia atrás (backward)** usando el modelo de regresión lineal.  
El proceso inicia con las variables del método forward y va eliminando las que menos aportan, buscando quedarse entre 2 y 5 características que maximicen el **R²**.  
Como resultado, las variables seleccionadas fueron: `acidezVolatil`, `cloruros`, `dioxidoAzufreTotal`, `sulfatos` y `alcohol`.


In [31]:
sfs_backward = SFS(
    estimator=lr,
    k_features=(2,5),
    forward=False,
    scoring='r2',
    cv=10
)

sfs_backward = sfs_backward.fit(X_train[feat_forward].values, Y_train.values)
feat_backward = [feat_forward[i] for i in sfs_backward.k_feature_idx_]

print("Variables seleccionadas con backward:", feat_backward)

Variables seleccionadas con backward: ['acidezVolatil', 'cloruros', 'dioxidoAzufreTotal', 'sulfatos', 'alcohol']


**6.** En este bloque se entrenaron dos modelos finales, uno con las variables seleccionadas por el método forward y otro con las del método backward.  
Se usó `.transform` para reducir las matrices de entrenamiento y prueba a las variables elegidas en cada caso, y se calculó el **R²** tanto en entrenamiento como en prueba.  
Los resultados muestran que el modelo forward obtuvo un R² en prueba de **0.4913**, mientras que el modelo backward alcanzó un R² de **0.3959**.  
Esto indica que, aunque backward utiliza menos variables, el modelo forward ofrece un mejor poder predictivo en los datos de prueba.


In [29]:
Xtr_fwd = sfs_forward.transform(X_train.values)
Xte_fwd = sfs_forward.transform(X_test.values)

lr.fit(Xtr_fwd, Y_train)
r2_train_fwd = r2_score(Y_train, lr.predict(Xtr_fwd))
r2_test_fwd  = r2_score(Y_test,  lr.predict(Xte_fwd))

sfs_backward = sfs_backward.fit(X_train[feat_forward].values, Y_train.values)
Xtr_bwd = sfs_backward.transform(X_train[feat_forward].values)
Xte_bwd = sfs_backward.transform(X_test[feat_forward].values)

lr.fit(Xtr_bwd, Y_train)
r2_train_bwd = r2_score(Y_train, lr.predict(Xtr_bwd))
r2_test_bwd  = r2_score(Y_test,  lr.predict(Xte_bwd))

print("R² test (forward): ", r2_test_fwd)
print("R² test (backward):", r2_test_bwd)

R² test (forward):  0.40126288354402995
R² test (backward): 0.3958889666765405


### Conclusión

En los resultados obtenidos, el modelo con selección **forward** alcanzó un R² en los datos de prueba de **0.4012**, mientras que el modelo con selección **backward** obtuvo un R² de **0.3959**.  

Aunque el método backward utilizó menos variables, su capacidad de predicción fue menor en comparación con el modelo forward. Por lo tanto, considero que el modelo **forward** es mejor en este caso, ya que logra un mayor poder explicativo sobre la calidad del vino en los datos de prueba, incluso si implica usar un número mayor de variables.